[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/HyperCoast/blob/main/docs/examples/pace_cyano.ipynb)

# Mapping Cyanobacteria with PACE data

## Install packages

Uncomment the following cell to install the HyperCoast package.

In [ ]:
# %pip install -U "hypercoast[extra]"

## Import libraries

In [ ]:
import earthaccess
import hypercoast
from hypercoast.pace import (
    cyano_band_ratios,
    apply_kmeans,
    apply_pca,
    apply_sam,
    apply_sam_spectral,
)

## Download PACE data

To download and access the PACE AOP data, you will need to create an Earthdata login. You can register for an account at [urs.earthdata.nasa.gov](https://urs.earthdata.nasa.gov). Once you have an account, run the following cell and enter your NASA Earthdata login credentials.

In [ ]:
earthaccess.login(persist=True)

Search for PACE AOP data:

In [ ]:
results = hypercoast.search_pace(
    bounding_box=(-83, 25, -81, 28),
    temporal=("2024-07-30", "2024-08-15"),
    short_name="PACE_OCI_L2_AOP_NRT",
    count=1,
)

Download PACE AOP data:

In [ ]:
hypercoast.download_pace(results[:1], out_dir="data")

## Read PACE data

Read PACE AOP data as an `xarray.Dataset`:

In [ ]:
filepath = "data/PACE_OCI.20240730T181157.L2.OC_AOP.V2_0.NRT.nc"
dataset = hypercoast.read_pace(filepath)
# dataset

![image](https://github.com/user-attachments/assets/56b1fae3-9acf-4ee1-8dc9-7f6784bedf88)

## Compute band ratios

In [ ]:
da = cyano_band_ratios(dataset, plot=True)

## The spectra of cyanobacteria bloom:

![](https://i.imgur.com/pQP50bz.png)

## Cyanobacteria and Spectral Angle Mapper

Spectral Angle Mapper: Spectral similarity
Input: library of Cyanobacteria bloom Rrs spectra with Chla at different levels

Spectral Mixture Analysis: unmix different cyanobacteria species based on spectral difference.

![](https://i.imgur.com/xLaLMA4.png)

## K-means applied to the whole image

In [ ]:
cluster_labels, latitudes, longitudes = apply_kmeans(dataset, n_clusters=6)

## K-means applied to selected pixels

In [ ]:
da = dataset["Rrs"]

filter_condition = (
    (da.sel(wavelength=650) > da.sel(wavelength=620))
    & (da.sel(wavelength=701) > da.sel(wavelength=681))
    & (da.sel(wavelength=701) > da.sel(wavelength=450))
)
extent = [-95, -85, 27, 33]
colors = ["#e41a1c", "#377eb8", "#4daf4a", "#f781bf", "#a65628", "#984ea3"]

cluster_labels, latitudes, longitudes = apply_kmeans(
    da, n_clusters=6, filter_condition=filter_condition, extent=extent, colors=colors
)

## Principal Component Analysis (PCA)

In [ ]:
pca_data = apply_pca(dataset, n_components=3, x_component=0, y_component=1)

In [ ]:
pca_data = apply_pca(dataset, n_components=3, x_component=1, y_component=2)

## Spectral Angle Mapper (SAM)

### Apply SAM to the whole image

In [ ]:
data, latitudes, longitudes = apply_sam(
    dataset,
    n_components=3,
    n_clusters=6,
)

### Apply SAM to selected pixels

In [ ]:
extent = [-95, -85, 27, 33]
colors = ["#377eb8", "#ff7f00", "#4daf4a", "#f781bf", "#a65628", "#984ea3"]
data, latitudes, longitudes = apply_sam(
    dataset,
    n_components=3,
    n_clusters=6,
    extent=extent,
    colors=colors,
)

### Apply SAM with a filtering condition

In [ ]:
da = dataset["Rrs"]

filter_condition = (
    (da.sel(wavelength=650) > da.sel(wavelength=620))
    & (da.sel(wavelength=701) > da.sel(wavelength=681))
    & (da.sel(wavelength=701) > da.sel(wavelength=450))
)
extent = [-95, -85, 27, 33]
colors = ["#e41a1c", "#377eb8", "#4daf4a", "#f781bf", "#a65628", "#984ea3"]

data, latitudes, longitudes = apply_sam(
    dataset,
    n_components=3,
    n_clusters=6,
    filter_condition=filter_condition,
    extent=extent,
    colors=colors,
)

### Use spectral library

In [ ]:
filepath = "data/PACE_OCI.20240730T181157.L2.OC_AOP.V2_0.NRT.nc"
dataset = hypercoast.read_pace(filepath)
url = "https://github.com/opengeos/datasets/releases/download/hypercoast/SAM_spectral_library.zip"
hypercoast.download_file(url)
spectral_library = "./SAM_spectral_library/*.csv"

In [ ]:
extent = [-95, -85, 27, 33]
data, latitudes, longitudes = apply_sam_spectral(
    dataset,
    spectral_library=spectral_library,
    extent=extent,
)

In [ ]:
da = dataset["Rrs"]
extent = [-95, -85, 27, 33]
filter_condition = (
    (da.sel(wavelength=650) > da.sel(wavelength=620))
    & (da.sel(wavelength=701) > da.sel(wavelength=681))
    & (da.sel(wavelength=701) > da.sel(wavelength=450))
)
data, latitudes, longitudes = apply_sam_spectral(
    da,
    spectral_library=spectral_library,
    filter_condition=filter_condition,
    extent=extent,
)